# A2A (Agent-to-Agent) 통신 시작하기

이 노트북은 에이전트 간 통신을 위한 표준화된 방법인 A2A 프로토콜을 소개합니다.

## 구축할 시스템

1. **여행지 추천 에이전트** - 유저 쿼리에 맞는 여행지 및 음식, 활동을 추천
2. **경로 최적화 에이전트** - 여행 경로를 최적화
3. **호스트 에이전트** - 다른 에이전트들을 순차적으로 조율

## 기타 리소스

- [Google ADK Documentation](https://google.github.io/adk-docs/)
- [A2A Protocol Specification](https://github.com/google/a2a)

#### 중요!
A2A는 현재 개발 중(WIP)이므로, 현재 최신 버전의 adk, a2a는 이 코드의 내용과 달라질 수 있습니다.

### 설정 및 설치

먼저 필요한 의존성을 설치합니다:

In [ ]:
# # 필요한 패키지 설치
# import sys
# !{sys.executable} -m pip install -q google-adk==1.21.0 a2a-sdk==0.3.19 a2a==0.44
# !{sys.executable} -m pip install -q python-dotenv aiohttp uvicorn requests mermaid-python nest-asyncio litellm

## 1. A2A 소개

### Agent-to-Agent (A2A) 통신이란?

A2A는 AI 에이전트들이 다음을 수행할 수 있도록 하는 표준화된 프로토콜입니다:
- 서로의 기능을 **발견**
- 복잡한 작업을 해결하기 위해 **협력**
- 실시간 상호작용을 위한 응답 **스트리밍**

### 환경 설정

In [ ]:
from datetime import datetime
import logging

# 로깅 설정 - 파일과 콘솔 모두에 출력
log_filename = f"a2a_travel_agents_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"

# 루트 로거 설정
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)  # 모든 레벨의 로그 캡처

# 로그 포맷 설정
formatter = logging.Formatter(
    "%(asctime)s - %(levelname)s - %(name)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)

# 파일 핸들러 - 모든 로그를 파일에 저장
file_handler = logging.FileHandler(log_filename, encoding="utf-8")
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# 콘솔 핸들러 - WARNING 이상만 콘솔에 출력
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.WARNING)
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# 시작 로그
logger.info(f"로그 파일 생성: {log_filename}")
logger.info("A2A 여행 계획 에이전트 서버 초기화 시작")

In [ ]:

import sys

from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver


class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [ ]:

import asyncio
import logging
import os
import sys
import threading
import time

from typing import Any

import httpx
import nest_asyncio
import uvicorn

from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.adk.models.lite_llm import LiteLlm
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

## 2. A2A 시스템 구축

3개의 에이전트 시스템을 단계별로 구축해 봅시다:

1. **여행지 추천 에이전트** - 유저 쿼리에 맞는 여행지 및 음식, 활동을 추천
2. **경로 최적화 에이전트** - 여행 경로를 최적화
3. **호스트 에이전트** - 다른 에이전트들을 순차적으로 조율

### 에이전트 1: 여행지 추천 에이전트

이 에이전트는 여행지를 추천하는 에이전트입니다.

In [ ]:
destination_recommender = Agent(
    model=LiteLlm(model="azure/gpt-4.1"),
    name="destination_recommender_agent",
    instruction="""
    당신은 전문 여행 가이드입니다. 사용자가 요청한 도시에 대해 반드시 방문해야 할 장소를 추천하는 것이 주요 업무입니다.

    사용자가 도시 이름을 제공하면:
    1. 해당 도시의 유명 관광지 3곳 추천
    2. 꼭 먹어봐야 할 현지 음식 2가지 추천
    3. 추천하는 현지 활동 2가지 추천

    반드시 다음 JSON 형식으로 응답해야 합니다:
    {
        "city": "도시 이름",
        "attractions": [
            {
                "name": "관광지 이름",
                "description": "간단한 설명 (1-2 문장)",
                "estimated_time": "예상 소요 시간 (예: 2시간)"
            },
            {
                "name": "관광지 이름",
                "description": "간단한 설명 (1-2 문장)",
                "estimated_time": "예상 소요 시간"
            },
            {
                "name": "관광지 이름",
                "description": "간단한 설명 (1-2 문장)",
                "estimated_time": "예상 소요 시간"
            }
        ],
        "foods": [
            {
                "name": "음식 이름",
                "description": "간단한 설명",
                "typical_meal_time": "추천 식사 시간 (예: 점심, 저녁)"
            },
            {
                "name": "음식 이름",
                "description": "간단한 설명",
                "typical_meal_time": "추천 식사 시간"
            }
        ],
        "activities": [
            {
                "name": "활동 이름",
                "description": "간단한 설명",
                "duration": "소요 시간"
            },
            {
                "name": "활동 이름",
                "description": "간단한 설명",
                "duration": "소요 시간"
            }
        ]
    }

    중요: JSON 객체만 반환하고 추가 텍스트나 마크다운 코드 블록(```)은 절대 포함하지 마세요.
    순수 JSON만 출력하세요.
    """,
    tools=[],  # Tool 없음 - LLM의 내장 지식만 사용
)
print("여행지 추천 에이전트 생성 완료")

In [ ]:
destination_recommender_card = AgentCard(
    name="여행지 추천 에이전트",
    url="http://localhost:10020",
    description="도시에 대한 관광지, 음식, 활동을 추천합니다",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=["text/plain"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="recommend_destinations",
            name="여행지 추천",
            description="특정 도시의 관광지, 음식, 활동을 추천합니다",
            tags=["여행", "관광", "음식", "활동"],
            examples=[
                "파리에서 방문할 곳을 추천해주세요",
                "도쿄의 명소를 알려주세요",
                "뉴욕에서 무엇을 해야 하나요?",
            ],
        )
    ],
)
print("여행지 추천 에이전트 카드 생성 완료")

In [ ]:
remote_recommender = RemoteA2aAgent(
    name="recommend_destinations",
    description="도시에 대한 관광지, 음식, 활동을 추천합니다",
    agent_card=f"http://localhost:10020{AGENT_CARD_WELL_KNOWN_PATH}",
)

### 에이전트 2: 일정 최적화 에이전트

추천받은 관광지 및 활동 등을 바탕으로 최적의 일정을 구성합니다.

In [ ]:
print("일정 최적화 에이전트 생성 시작...")
itinerary_optimizer = Agent(
    model=LiteLlm(model="azure/gpt-4.1"),
    name="itinerary_optimizer_agent",
    instruction="""
    당신은 여행 일정 최적화 전문가입니다. 추천받은 관광지, 음식, 활동을 바탕으로
    하루 일정을 효율적으로 구성하는 것이 주요 업무입니다.

    입력으로 JSON 형식의 추천 정보가 주어지면:
    1. 지리적 위치와 이동 시간을 고려하여 동선 최적화
    2. 식사 시간을 적절히 배치
    3. 시간대별로 활동을 배치 (오전/오후/저녁)
    4. 각 활동 간 이동 시간 포함

    일정은 오전 9시부터 시작하여 저녁 9시까지 구성하세요.
    이동 시간은 20-30분으로 가정하세요.
    점심은 12:00-13:00, 저녁은 18:00-19:00 사이에 배치하세요
    """,
    tools=[],  # Tool 없음 - LLM의 내장 지식만 사용
)
print("일정 최적화 에이전트 생성 완료")

In [ ]:
itinerary_optimizer_card = AgentCard(
    name="일정 최적화 에이전트",
    url="http://localhost:10021",
    description="추천받은 장소들을 효율적인 하루 일정으로 구성합니다",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=["application/json"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="optimize_itinerary",
            name="일정 최적화",
            description="추천받은 관광지, 음식, 활동을 하루 일정으로 최적화합니다",
            tags=["일정", "최적화", "동선", "시간관리"],
            examples=[
                "이 추천들을 하루 일정으로 만들어주세요",
                "효율적인 여행 동선을 짜주세요",
                "시간대별 일정을 구성해주세요",
            ],
        )
    ],
)
print("일정 최적화 에이전트 카드 생성 완료")
print("기본 에이전트 초기화 완료")

In [ ]:
remote_optimizer = RemoteA2aAgent(
    name="optimize_itinerary",
    description="추천받은 장소들을 효율적인 하루 일정으로 구성합니다",
    agent_card=f"http://localhost:10021{AGENT_CARD_WELL_KNOWN_PATH}",
)

### 에이전트 3: 호스트 에이전트 (오케스트레이터)

호스트 에이전트는 다른 두 에이전트 사이를 조율하여 종합적인 트렌드 분석을 제공합니다.

In [ ]:
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH

host_agent = SequentialAgent(
    name="travel_planner_host",
    sub_agents=[remote_recommender, remote_optimizer],
)

host_agent_card = AgentCard(
    name="여행 계획 호스트",
    url="http://localhost:10022",
    description="여행지 추천 및 일정 최적화를 순서대로 수행합니다",
    version="1.0",
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=["text/plain"],
    default_output_modes=["application/json"],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[
        AgentSkill(
            id="complete_travel_plan",
            name="완전한 여행 계획",
            description="도시명을 입력받아 추천과 최적화된 일정을 모두 제공합니다.",
            tags=["여행계획", "조율", "종합", "일정"],
            examples=[
                "파리 여행 계획을 짜주세요",
                "서울 하루 여행 일정을 만들어주세요",
                "런던 여행 계획을 세워주세요",
            ],
        )
    ],
)

## 3. 실행

이제 모든 것을 통합해 봅시다. 에이전트를 시작하고 전체 시스템을 실행하기 위한 헬퍼 함수를 만들겠습니다.

### A2A 서버 시작

각 에이전트를 A2A 서버로 실행하는 함수 생성:

In [ ]:
def create_agent_a2a_server(agent, agent_card):
    """모든 ADK 에이전트를 위한 A2A 서버를 생성합니다.

    Args:
        agent: ADK 에이전트 인스턴스
        agent_card: ADK 에이전트 카드

    Returns:
        A2AStarletteApplication 인스턴스
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # A2A 애플리케이션 생성
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [ ]:

# nest_asyncio 적용
nest_asyncio.apply()

# 서버 태스크 저장
server_tasks: list[asyncio.Task] = []


async def run_agent_server(agent, agent_card, port) -> None:
    """단일 에이전트 서버를 실행합니다."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',  # 중요: uvicorn이 현재 루프를 사용하도록 함
    )

    server = uvicorn.Server(config)
    await server.serve()


async def start_all_servers() -> None:
    """동일한 이벤트 루프에서 모든 서버를 시작합니다."""
    # 모든 서버를 위한 태스크 생성
    tasks = [
        asyncio.create_task(
            run_agent_server(destination_recommender, destination_recommender_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(itinerary_optimizer, itinerary_optimizer_card, 10021)
        ),
        asyncio.create_task(
            run_agent_server(host_agent, host_agent_card, 10022)
        ),
    ]

    # 서버 시작 시간 대기
    await asyncio.sleep(2)

    print('✅ 모든 에이전트 서버가 시작되었습니다!')
    print('   - 트렌딩 에이전트: http://127.0.0.1:10020')
    print('   - 분석 에이전트: http://127.0.0.1:10021')
    print('   - 호스트 에이전트: http://127.0.0.1:10022')

    # 서버를 계속 실행
    try:
        await asyncio.gather(*tasks)
    except KeyboardInterrupt:
        print('서버를 종료하는 중...')


# 백그라운드 스레드에서 실행


def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())


# 스레드 시작
server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

# 서버가 준비될 때까지 대기
time.sleep(3)

In [ ]:
# 서버가 실행 중인지 확인
import requests

try:
    response = requests.get('http://127.0.0.1:10020/.well-known/agent-card.json', timeout=5)
    if response.status_code == 200:
        print(response.content)
        print('여행지 추천 에이전트 서버 응답 확인')
    else:
        print(f'⚠️  서버 응답 상태 코드: {response.status_code}')
except Exception as e:
    print(f'❌ 서버 연결 실패: {e}')
    print('   서버가 아직 시작 중일 수 있습니다. 잠시 후 다시 시도하세요.')

## 4. 시스템 테스트

### A2A 에이전트 호출 (2개의 원격 에이전트와 이들을 서브 에이전트로 참조하는 호스트 에이전트)

In [ ]:

class A2ASimpleClient:
    """A2A 서버를 호출하는 간단한 A2A 클라이언트."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # 에이전트 메타데이터 캐시
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """공식 A2A SDK 패턴을 따라 메시지를 전송합니다."""
        # 타임아웃으로 httpx 클라이언트 설정
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # 캐시된 에이전트 카드 데이터가 있는지 확인
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # 에이전트 카드 가져오기
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # 데이터로부터 AgentCard 생성
            agent_card = AgentCard(**agent_card_data)

            # 에이전트 카드로 A2A 클라이언트 생성
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # 메시지 객체 생성
            message_obj = create_text_message_object(content=message)

            # 메시지를 보내고 응답 수집
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # 응답은 튜플 - 첫 번째 요소(Task 객체) 가져오기
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # 튜플의 첫 번째 요소

                # 텍스트 추출: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return '응답을 받지 못했습니다'

In [ ]:
# 클라이언트 초기화
a2a_client = A2ASimpleClient()

In [ ]:
async def test_destination_recommender() -> None:
    """여행지 추천 에이전트를 테스트합니다."""
    destination_recommended = await a2a_client.create_task(
        'http://localhost:10020', "경주에는 볼만한게 있나요?"
    )
    print(destination_recommended)


# 비동기 함수 실행
asyncio.run(test_destination_recommender())

In [ ]:
async def test_itinerary_optimizer() -> None:
    """분석 에이전트를 테스트합니다."""
    itinerary = await a2a_client.create_task(
        'http://localhost:10021', '경주 여행 경로를 추천해주세요'
    )
    print(itinerary)

asyncio.run(test_itinerary_optimizer())

In [ ]:
async def test_host_analysis() -> None:
    """호스트 에이전트를 테스트합니다."""
    host_analysis = await a2a_client.create_task(
        'http://localhost:10022',
        '삼척 여행 계획을 세워주세요',
    )
    print("=" * 100)
    print(host_analysis)


asyncio.run(test_host_analysis())

## 요약

축하합니다! A2A 프로토콜을 사용하여 멀티 에이전트 시스템을 성공적으로 구축했습니다.  
이 실습을 통해서 살펴본 내용을 정리하면 아래와 같습니다.  

1. **A2A 프로토콜 기초**: 에이전트가 서로를 발견하고 통신하는 방법
2. **ADK 통합**: ADK 에이전트를 만들고 A2A용으로 래핑하기
3. **에이전트 조율**: 여러 에이전트를 조율하는 호스트 에이전트 구축
4. **실제 구현**: 완전한 멀티 에이전트 시스템 실행 및 테스트